In [ ]:
# %load option_chain_for_3dot6.py


import csv
from bs4 import BeautifulSoup
import requests

index_symbol = ('NIFTY','JUNIOR','CNX100','CNX200','CNX500','NIFTYMID50','CNXMIDCAP','CNXSMALLCAP',
                        'NIFTYDIV','CNXAUTO','BANKNIFTY','CNXENERGY','CNXFIN','CNXFMCG','CNXIT','CNXMEDIA',
                        'CNXMETAL','CNXMNC','CNXPHARMA','CNXPSUBANK','CNXREALTY','CNXCONS','CNXCOMM','CNXDIVOPP',
                        'CNXINFRA','CNXPSE','CNXSERVICE','INDIAVIX','LIX15') 
def first_substring(strings, substring):
    return next(i for i, string in enumerate(strings) if substring in string)

def getFNOsymbols():
    url = 'https://www.nseindia.com/content/fo/fo_mktlots.csv'
    with requests.Session() as s:
        download = s.get(url)
    
        decoded_content = download.content.decode('utf-8')
    
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        lol = list(cr)

    del lol[first_substring(lol,'Derivatives on Individual Securities')]
    del lol[0]
    symbols=[]
    ls={}
    for line in lol:
        sym=line[1].strip()
        size=line[2].strip()
        if size!='':
            symbols.append(sym)
            ls[sym]=int(size)
    return symbols,ls


fnosymbols,lotsizes=getFNOsymbols()
expiry='29AUG2019'
symbol='NIFTY'

symbol=input('Enter : ')

while (symbol != 'exit') :

    if symbol in index_symbol:
        instr='OPTIDX'
    else:
        instr='OPTSTK'    
    symbol1=''
    for i in range(0,len(symbol)):
    	if symbol[i]=='&':
    	    symbol1=symbol1+'%26'
    	else:
    	    symbol1=symbol1+str(symbol[i])  

    site= "http://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?segmentLink=17&instrument="+instr+"&symbol="+symbol1+"&date="+expiry
#    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
#                           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#                           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
#                           'Accept-Encoding': 'none',
#                           'Accept-Language': 'en-US,en;q=0.8',
#                           'Connection': 'keep-alive'}
#    
#    req = urllib2.Request(site, headers=hdr)
    """
    page = urllib2.urlopen(req)
    soup = BeautifulSoup()
    soup = BeautifulSoup(page,"lxml")
    """
    r=requests.get(site)
    soup=BeautifulSoup(r.content, 'html5lib') 
    table = soup.find_all(id='octable')
    trs = table[0].find_all('tr')
    trs=trs[2:-1]
    
    cl=[]
    pl=[]
    csp=[]
    psp=[]
    for tr in trs:
    	tds = tr.find_all('td')
        
    	csp.append(float(tds[11].get_text().strip().replace(',','')))
        
    	list1=['oi','coi','vol','iv','ltp','nc','bq','bp','ap','aq']
    	list2=[tds[1].get_text().strip().replace(',',''),
            tds[2].get_text().strip().replace(',',''),  
            tds[3].get_text().strip().replace(',',''),
            tds[4].get_text().strip().replace(',',''),
            tds[5].get_text().strip().replace(',',''),
            tds[6].get_text().strip().replace(',',''),    
            tds[7].get_text().strip().replace(',',''),
            tds[8].get_text().strip().replace(',',''),  
            tds[9].get_text().strip().replace(',',''),
            tds[10].get_text().strip().replace(',','')]
    	l12=zip(list1,list2)
    	ccl=dict(l12)
    	cl.append(ccl)
    	
    	psp.append(float(tds[11].get_text().strip().replace(',','')))
        
    	list3=['oi','coi','vol','iv','ltp','nc','bq','bp','ap','aq']
    	list4=[tds[21].get_text().strip().replace(',',''),
            tds[20].get_text().strip().replace(',',''),
            tds[19].get_text().strip().replace(',',''),
            tds[18].get_text().strip().replace(',',''),
            tds[17].get_text().strip().replace(',',''),
            tds[16].get_text().strip().replace(',',''),    
            tds[12].get_text().strip().replace(',',''),
            tds[13].get_text().strip().replace(',',''),  
            tds[14].get_text().strip().replace(',',''),
            tds[15].get_text().strip().replace(',','')]
    	l34=zip(list3,list4)
    	ppl=dict(l34)
    	pl.append(ppl)
	
    dcl=zip(csp,cl)
    dpl=zip(psp,pl)
    call_op=dict(dcl)
    put_op=dict(dpl)
    t1=['call','put']
    d1=[call_op,put_op]
    t2=zip(t1,d1)
    xd=dict(t2)
    oi='oi'
    coi='coi'
    vol='vol'
    iv='iv'
    ltp='ltp'
    nc='nc'
    bq='bq'
    bp='bp'
    ap='ap'
    aq='aq'
    call='call'
    put='put'
     
    co=xd[call]
    po=xd[put]
    print ('call options')
    for key in co.keys():
    	if co[key][coi]!='-':
    	    print (key,co[key][ltp],co[key][iv],co[key][oi],co[key][coi])
    print ('put options')
    for key in po.keys():
    	if po[key][coi]!='-':
    	    print (key,po[key][ltp],po[key][iv],po[key][oi],po[key][coi])    
	    
    symbol=input('Enter symbol : ')

pass

Enter : HDFC
call options
2000.0 196.95 - 33500 -500
2040.0 177.25 34.42 5000 500
2060.0 141.45 18.31 15500 -4000
2080.0 122.90 17.67 20000 -3500
2100.0 108.60 19.91 97000 -4000
2120.0 92.10 19.39 41500 -10000
2140.0 80.00 20.74 72500 -39000
2160.0 67.00 20.77 54000 -49000
2180.0 55.00 20.65 84500 -9000
2200.0 44.10 20.40 242000 -10000
2220.0 35.60 20.61 87000 15500
2240.0 26.45 19.87 82000 19000
2260.0 21.00 20.28 82000 18000
2280.0 14.85 19.68 69000 29500
2300.0 12.40 20.68 175500 -24000
2320.0 10.90 20.59 25000 5500
2340.0 6.00 20.32 55500 9000
2360.0 4.80 20.81 29000 500
2380.0 3.50 21.00 25500 1000
2400.0 2.50 21.13 123000 7000
2480.0 1.00 23.34 15000 500
2500.0 1.05 24.83 41500 500
put options
1800.0 1.00 36.91 16500 -1500
1900.0 1.85 31.00 61500 1500
1960.0 3.00 27.83 23500 -3500
2000.0 4.75 26.43 305500 3000
2020.0 6.50 26.31 39500 -7000
2040.0 7.75 25.26 70000 -10500
2060.0 9.90 24.72 60000 5000
2080.0 13.45 24.80 44000 -4000
2100.0 15.45 23.36 204000 73500
2120.0 19.85 23.03 